<a href="https://colab.research.google.com/github/IDF13/mulcam_army/blob/sumin/Aritsts_Textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# -*- coding: utf-8 -*-
"""k-means_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Ls1XFeatzdMivoFijtX0MuN9E7ADviAC
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

"""# 데이터 불러오기
댓글을 얼마나 불러와야 할까
"""


# 아티스트 51팀의 데이터 불러옴 (전처리가 잘 되지 않은 7개 아티스트 제외)
import os

path = '/content/drive/MyDrive/[공유] Mulcam_Army 공유폴더!/크롤링 한 자료/k-pop_Radar아티스트 크롤링 할당분/수민_결과물/전처리 완료/'
file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.csv')] ## 파일명 끝이 .csv인 경우
# https://boleumdal.tistory.com/entry/python-%ED%8F%B4%EB%8D%94-%EC%95%88%EC%97%90-\
# %EC%9E%88%EB%8A%94-%ED%8C%8C%EC%9D%BC-%ED%95%9C%EB%B2%88%EC%97%90-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0


# csv 파일들을 DataFrame으로 불러와서 concat
df = pd.DataFrame()
for i in file_list:
    data = pd.read_csv(path + i, encoding='utf-8', header=None, engine='python')
    df = pd.concat([df,data])
    
df = df.reset_index(drop=True)
df.dropna(axis=0)

,0,1,2
0,even if you re not a carat let s all admit it ...,28000.0,(en)
1,this song tells that some song s don t need bi...,1900.0,(en)
2,외국분들 댓글 너무 착하셔세봉이들이 보고 감동했으면 좋겠네요,72.0,(ko)
3,not a fan but i should say that this song is o...,625.0,(en)
4,4 years have passed and this song is still a l...,96.0,(en)
...,...,...,...
2319905,or the rapper,0.0,(en)
2319906,yes yes,0.0,(en)
2319907,the guy in the middle,0.0,(en)
2319908,victon 00 11 han seungwoo [ leader rapper voca...,0.0,(en)


In [10]:
df.columns = ['comment','like','lang']
# len(df['comment'])
df.head()

,comment,like,lang
0,even if you re not a carat let s all admit it ...,28000.0,(en)
1,this song tells that some song s don t need bi...,1900.0,(en)
2,외국분들 댓글 너무 착하셔세봉이들이 보고 감동했으면 좋겠네요,72.0,(ko)
3,not a fan but i should say that this song is o...,625.0,(en)
4,4 years have passed and this song is still a l...,96.0,(en)


In [11]:
data_ko = pd.DataFrame([kor[:1] for kor in df.values if kor[2] == '(ko)'], columns=['comment'])
data_en = pd.DataFrame([en[:1] for en in df.values if en[2] == '(en)'], columns=['comment'])
data_en.comment
len(data_en)

1803300

In [12]:
data_en['comment']

0          even if you re not a carat let s all admit it ...
1          this song tells that some song s don t need bi...
2          not a fan but i should say that this song is o...
3          4 years have passed and this song is still a l...
4          i m not a fan of them but i really like all of...
                                 ...                        
1803295                                        or the rapper
1803296                                              yes yes
1803297                                the guy in the middle
1803298    victon 00 11 han seungwoo [ leader rapper voca...
1803299                                       omg thanks fam
Name: comment, Length: 1803300, dtype: object

In [13]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re

In [14]:
# 데이터 랜덤으로 추출(10만개 댓글)
import random
import numpy as np

# a = data_en.comment[:50000]
# z = random.shuffle(a)
# len(z)

a = np.random.choice(data_en.comment.values, 100000, replace = False)

In [15]:
a

array(['welcome uwu if you need any help i d be happy to help',
       'i feel so energetic listening to this song gfriend < 3',
       'this is a reason to wake up lol', ..., 'in your dream',
       'ong yung shin well we didn t say anything about snsd doing plastic surgery we just simply said our opinions on why snsd still dominates the kpop industry and it is due to the fans they ve gained in 9 yrs we did not say anything about one group being more popular due to their looks but as a panda i would still think that my girls are the most beautiful girl group ever lol and the comparison you wrote up their doesn t give your idols any justice by comparing their looks to after school like come on dude this isn t a beauty pageant this is the kpop industry you don t need to be beautiful to have talent or be an amazing singer like eunji when she first started her manager found her from a talent show and she also won first place on the kbs singing show as a child now that s what i mean by tal

In [16]:
a_df = pd.DataFrame(a)

In [17]:
a_df.columns = ['comment']
a_df

,comment
0,welcome uwu if you need any help i d be happy ...
1,i feel so energetic listening to this song gfr...
2,this is a reason to wake up lol
3,24 in belize but its great tho hope more peopl...
4,ii s so sad i lost some thing
...,...
99995,ness can you recommend me which iph colour sho...
99996,ong yung shin well we didn t say anything abou...
99997,in your dream
99998,ong yung shin well we didn t say anything abou...


In [18]:
# boyz, gurl

In [19]:
# for i in range(len(data_en.comment)):
#     data_en.comment[i] = str(data_en.comment[i])

for i in range(len(a_df.comment)):
    a_df.comment[i] = str(a_df.comment[i])

# 숫자제거 / 밑줄 제외한 특수문자 제거
p = re.compile("[0-7]+")
z = re.compile("[8-9]+")
q = re.compile("\W+")
r = re.compile('[^a-zA-Z]+')

en = []
for i in a:
    tokens = re.sub(p," ",i)
    tokens = re.sub(z," ",tokens)
    tokens = re.sub(q," ",tokens)
    tokens = re.sub(r," ", tokens)
    en.append(tokens)
len(en)
en[:10]

# data_ens =  data_en['comment'].sample(
#     # n = '50000',
#     frac ='0.3',
#     replace = False, #복원/비복원 여부. 동일한 행의 샘플링을 두 번 이상 허용할 것인가
#     random_state = 1
# )

['welcome uwu if you need any help i d be happy to help',
 'i feel so energetic listening to this song gfriend ',
 'this is a reason to wake up lol',
 ' in belize but its great tho hope more people give them recognition',
 'ii s so sad i lost some thing',
 'jennie the mandu no b views no opinion sorry that itzy is beter than you',
 'i found youuu yeah',
 'let s make it',
 'even saying thank you minute by minute second by second is not enough to show you how grateful and proud we are to know you and your music you ve done soooo much things for us and i know you ll do even greater in the future right now although this isn t enough i still want to say thank you your music made such a great impact to me as a person it contributed a lot to the person i am today seventeen thank you for being home to many people',
 'i m so excited for next comeback']

In [20]:
# 불용어 제거

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english')) 
stop_words.update(('gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc'))
#(('song','group','songs','youtube','views','time','https','girl','girls','people','yes','lol','video','part','member','members', 'look','way','guys','fans','fan'))

# # stopwords_gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc
# [('talent', 10422), ('group', 9234), ('comment', 8440), ('preach', 7575), ('gurl', 7573), ('show', 7363), ('anything', 6983), ('kpop', 6954), ('idols', 6882), ('industry', 6858), ('looks', 6854), ('song', 6812), ('lol', 6022), ('reason', 5355), ('wake', 5090), ('fans', 4325), ('girl', 4241), ('girls', 4172), ('members', 4138), ('x', 3830), ('work', 3826), ('xd', 3720), ('rapper', 3692), ('sm', 3674), ('school', 3650), ('place', 3582), ('mino', 3478), ('singer', 3462), ('beauty', 3437), ('etc', 3394)]


# for i in range(len(data_en.comment[:10000])):
#     data_en.comment[i] = str(data_en.comment[i])

res=[]
for i in range(len(en)):
    word_tokens = word_tokenize(en[i])

    result = []
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w) 
    res.append(result)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [21]:
# print(word_tokens) 
print(res[:10])
print(len(res))

en_pos = []
for i in range(len(res)):
    tokens_pos = nltk.pos_tag(res[i])
    en_pos.append(tokens_pos)

en_pos[:5]

[['welcome', 'uwu', 'need', 'help', 'happy', 'help'], ['feel', 'energetic', 'listening', 'song', 'gfriend'], ['reason', 'wake', 'lol'], ['belize', 'great', 'tho', 'hope', 'people', 'give', 'recognition'], ['ii', 'sad', 'lost', 'thing'], ['jennie', 'mandu', 'views', 'opinion', 'sorry', 'itzy', 'beter'], ['found', 'youuu', 'yeah'], ['let', 'make'], ['even', 'saying', 'thank', 'minute', 'minute', 'second', 'second', 'enough', 'show', 'grateful', 'proud', 'know', 'music', 'done', 'soooo', 'much', 'things', 'us', 'know', 'even', 'greater', 'future', 'right', 'although', 'enough', 'still', 'want', 'say', 'thank', 'music', 'made', 'great', 'impact', 'person', 'contributed', 'lot', 'person', 'today', 'seventeen', 'thank', 'home', 'many', 'people'], ['excited', 'next', 'comeback']]
100000


[[('welcome', 'JJ'),
  ('uwu', 'JJ'),
  ('need', 'NN'),
  ('help', 'NN'),
  ('happy', 'VB'),
  ('help', 'NN')],
 [('feel', 'VB'),
  ('energetic', 'JJ'),
  ('listening', 'NN'),
  ('song', 'NN'),
  ('gfriend', 'NN')],
 [('reason', 'NN'), ('wake', 'NN'), ('lol', 'NN')],
 [('belize', 'VB'),
  ('great', 'JJ'),
  ('tho', 'JJ'),
  ('hope', 'NN'),
  ('people', 'NNS'),
  ('give', 'VBP'),
  ('recognition', 'NN')],
 [('ii', 'NN'), ('sad', 'NN'), ('lost', 'VBD'), ('thing', 'NN')]]

In [22]:
# 명사는 NN을 포함하고 있음을 알 수 있음
en_NN=[]
for i in range(len(en_pos)):
    NN_words = []
    for word, pos in en_pos[i]:
        if 'NN' in pos:
            NN_words.append(word)
        elif 'NN' in pos:
            NN_words.append(word)
    en_NN.extend(NN_words)
len(en_NN)

516719

In [23]:
stop_words = set(stopwords.words('english')) 
stop_words.update(('gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc'))

res22=[]
for i in range(len(en_NN)):
    word_tokens = word_tokenize(en_NN[i])

    result = []
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w) 
    res22.append(result)
len(res22)

516719

In [24]:
stop_words = set(stopwords.words('english')) 
stop_words.update(('gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc'))

res11=[]
for w in en_NN: 
    if w not in stop_words: 
        res11.append(w) 

len(res11)

516719

In [25]:
# 형용사

en_ADJ=[]
for i in range(len(en_pos)):
    JJ_words = []
    for word, pos in en_pos[i]:
        if 'JJ' in pos:
            JJ_words.append(word)
        elif 'JJ' in pos:
            JJ_words.append(word)
    en_ADJ.extend(JJ_words)
en_ADJ[:10]
#len(en_ADJ) #96230

['welcome',
 'uwu',
 'energetic',
 'great',
 'tho',
 'itzy',
 'minute',
 'minute',
 'second',
 'second']

In [26]:
# 동사

en_VB=[]
for i in range(len(en_pos)):
    VB_words = []
    for word, pos in en_pos[i]:
        if 'VB' in pos:
            VB_words.append(word)
        elif 'VB' in pos:
            VB_words.append(word)
    en_VB.extend(VB_words)
en_VB[:10]
#len(en_VB) #95529

['happy',
 'feel',
 'belize',
 'give',
 'lost',
 'sorry',
 'found',
 'let',
 'make',
 'saying']

In [27]:
## 중복 삭제 

# en_NN_uni= pd.unique(en_NN)
# en_ADJ_uni= pd.unique(en_ADJ)
# en_VB_uni= pd.unique(en_VB)

In [28]:
# a = pd.concat([en_NN_uni,en_ADJ_uni,en_VB_uni])

# en_NN_df = pd.DataFrame(en_NN_uni)
# en_ADJ_df = pd.DataFrame(en_ADJ_uni)
# en_VB_df = pd.DataFrame(en_VB_uni)

# df_ADV = pd.DataFrame(en_ADV)
# a['df_ADJ_uni']
# pd.DataFrame(en_ADJ_uni)
# a['df_VB_uni'] 
# pd.DataFrame(en_VB_uni)

In [29]:
# # 문장 빈도

# from collections import Counter

# c = Counter(a) # input type should be a list of words (or tokens)
# k1 = 30
# print(c.most_common(k1)) # 빈도수 기준 상위 k개 단어 출력

In [30]:
# 명사 빈도(10만개)

from collections import Counter

c = Counter(en_NN) # input type should be a list of words (or tokens)
k1 = 30
print(c.most_common(k1)) # 빈도수 기준 상위 k개 단어 출력

[('talent', 10419), ('group', 9394), ('comment', 8371), ('preach', 7615), ('gurl', 7614), ('show', 7389), ('kpop', 7069), ('anything', 7002), ('idols', 6904), ('looks', 6874), ('industry', 6858), ('song', 6816), ('lol', 5964), ('reason', 5279), ('wake', 5033), ('girl', 4337), ('fans', 4302), ('girls', 4199), ('members', 4191), ('work', 3798), ('rapper', 3707), ('xd', 3704), ('sm', 3678), ('school', 3655), ('place', 3602), ('singer', 3498), ('mino', 3458), ('beauty', 3455), ('etc', 3426), ('yrs', 3387)]


In [31]:
# stopwords_gurl, girl, girls, members, song, xd, x, kpop, group, anything
# [('talent', 10422), ('group', 9234), ('comment', 8440), ('preach', 7575), ('gurl', 7573), ('show', 7363), ('anything', 6983), ('kpop', 6954), ('idols', 6882), ('industry', 6858), ('looks', 6854), ('song', 6812), ('lol', 6022), ('reason', 5355), ('wake', 5090), ('fans', 4325), ('girl', 4241), ('girls', 4172), ('members', 4138), ('x', 3830), ('work', 3826), ('xd', 3720), ('rapper', 3692), ('sm', 3674), ('school', 3650), ('place', 3582), ('mino', 3478), ('singer', 3462), ('beauty', 3437), ('etc', 3394)]

In [ ]:
# # stopwords

# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# stop_words = set(stopwords.words('english')) 
# stop_words.update(('gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc'))

# word_tokens = word_tokenize(example_sent)
 
# filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
# filtered_sentence = []
 
# for w in word_tokens:
#     if w not in stop_words:
#         filtered_sentence.append(w)
 
# print(word_tokens)
# print(filtered_sentence)

In [34]:
stop_words = set(stopwords.words('english')) 
stop_words.update(('gurl, girl, girls, members, song, xd, x, kpop, group, anything, etc'))

res22=[]
for i in range(len(en_NN)):
    word_tokens = word_tokenize(en_NN[i])

    result = []
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w) 
    res22.append(result)
len(res22)

516719

In [35]:
res=[]
for i in range(len(en)):
    word_tokens = word_tokenize(en[i])

    result = []
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w) 
    res.append(result)

In [36]:
# 3단어 이하 짧은 단어 제거
# remove words less than three letters

print(res[1])
for word in res[1]:
    print(word)

en_sent_less3=[]
for i in range(len(res)):
    tokens = [word for word in res[i] if len(word) >= 3]
    en_sent_less3.append(tokens)
en_sent_less3[:2]

['feel', 'energetic', 'listening', 'song', 'gfriend']
feel
energetic
listening
song
gfriend


[['welcome', 'uwu', 'need', 'help', 'happy', 'help'],
 ['feel', 'energetic', 'listening', 'song', 'gfriend']]

In [37]:
en_sent =[]
for i in range(len(en_sent_less3)):
    temp=" ".join(en_sent_less3[i])
    en_sent.append(temp)
en_sent[:15]

['welcome uwu need help happy help',
 'feel energetic listening song gfriend',
 'reason wake lol',
 'belize great tho hope people give recognition',
 'sad lost thing',
 'jennie mandu views opinion sorry itzy beter',
 'found youuu yeah',
 'let make',
 'even saying thank minute minute second second enough show grateful proud know music done soooo much things know even greater future right although enough still want say thank music made great impact person contributed lot person today seventeen thank home many people',
 'excited next comeback',
 'touchin heart since',
 'preach gurl best comment',
 'think one yeri',
 'bsides also bops check top secret hello yummy reality',
 'yes']

In [ ]:
!pip install text-summarizer

In [38]:
!pip install textrank

In [41]:
import textrank
from textrank import KeywordSummarizer

ImportError: ignored

In [40]:
def filter_tokenizer(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = filter_tokenizer,
    window = -1,
    verbose = False
)

keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
  print(word)


NameError: ignored

In [ ]:
 tokenize = nltk.tokenizer 

def scan_vocabulary(sents, tokenize, min_count=2):
  counter = Counter(w for sent in sents for w in komoran_tokenize(sent, tokenize))
  counter = {w:c for w,c in counter.items() if c >= min_count}
  idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
  vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
  return idx_to_vocab, vocab_to_idx

# 두 단어간의 유사도 계산
# co-occurrence는 문장 안에서 두 단어의 간격이 window인 횟수(2-8 굳)

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

# dict of dict 형식의 그래프를 scipy의 matrix로 변환
def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

# 그래프 그리기
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

# 만들어진 그래프에 pagerank 학습하는 함수 만들기
def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

# 함수 도출
def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [ ]:
textrank_keyword()

In [ ]:
# 핵심 문장 추출

def sent_graph(sents, tokenize, similarity, min_count=2, min_sim=0.3):
    _, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)

    tokens = [[w for w in tokenize(sent) if w in vocab_to_idx] for sent in sents]
    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

def textrank_keysentence(sents, tokenize, min_count, similarity, df=0.85, max_iter=30, topk=5):
    g = sent_graph(sents, tokenize, min_count, min_sim, similarity)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keysents = [(idx, R[idx], sents[idx]) for idx in reversed(idxs)]
    return keysents

In [ ]:
# 형용사 빈도

from collections import Counter

c = Counter(en_ADJ) # input type should be a list of words (or tokens)
k2 = 20
print(c.most_common(k2)) # 빈도수 기준 상위 k개 단어 출력

[('due', 10136), ('beautiful', 7441), ('many', 4454), ('first', 4337), ('popular', 3799), ('top', 3717), ('amazing', 3659), ('give', 3656), ('mean', 3624), ('cool', 3486), ('kpop', 3447), ('public', 3407), ('younger', 3403), ('f', 3397), ('snsd', 3374), ('shin', 3371), ('child', 3363), ('bi', 3362), ('versatile', 3360), ('looks', 3354)]


In [ ]:
# 동사 빈도

from collections import Counter

c = Counter(en_VB) # input type should be a list of words (or tokens)
k3 = 20
print(c.most_common(k3)) # 빈도수 기준 상위 k개 단어 출력

[('say', 7774), ('think', 4934), ('said', 4307), ('talented', 4151), ('come', 4119), ('need', 4062), ('started', 3709), ('found', 3697), ('singing', 3594), ('lol', 3527), ('agree', 3520), ('discovered', 3481), ('wrote', 3472), ('noticed', 3429), ('comparing', 3397), ('gained', 3388), ('grown', 3372), ('dude', 3367), ('teach', 3362), ('snsd', 3360)]


In [ ]:
# a = pd.concat([en_NN_uni,en_ADJ_uni,en_VB_uni])

a['df_NN_uni'] = pd.DataFrame(en_NN_uni)
# df_ADV = pd.DataFrame(en_ADV)
a['df_ADJ_uni'] = pd.DataFrame(en_ADJ_uni)
a['df_VB_uni'] = pd.DataFrame(en_VB_uni)

In [ ]:
a.columns = ['comment','like','lang']
# len(df['comment'])
df.head()

0             yeah i can feel your pain i tried to learn it ...
1             they looked so powerful standing on individual...
2                    i really miss hearing this song from class
3                                        phan yes it means very
4             it s been already 4 yrs since this song got re...
                                    ...                        
49998                                  gimbap kidding i m sorry
49999                       6 here in malaysia lets do it carat
df_NN_uni                   0
0          yeah
1          fee...
df_ADJ_uni                     0
0             lack
1       ...
df_VB_uni                      0
0            tried
1       ...
Name: comment, Length: 50003, dtype: object